# Proyek Klasifikasi Penumpang Titanic (KNN)

Notebook ini disusun untuk memenuhi tugas akhir mata kuliah Data Mining.
Tujuan proyek ini adalah memprediksi keselamatan penumpang kapal Titanic berdasarkan fitur seperti umur, kelas tiket, dan biaya, menggunakan algoritma **K-Nearest Neighbors (KNN)**.

## 1. Import Library
Langkah pertama adalah memanggil semua pustaka (library) Python yang dibutuhkan:
* **Pandas**: Untuk membaca dan mengolah data tabel.
* **Numpy**: Untuk operasi matematika.
* **Matplotlib & Seaborn**: Untuk membuat grafik visualisasi data.
* **Sklearn**: Library utama untuk algoritma Machine Learning (KNN, splitting data, evaluasi).

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, classification_report

## 2. Upload Dataset (Khusus Google Colab)
Jalankan cell di bawah ini untuk mengunggah file `titanic_clean.csv` dari komputermu ke dalam sesi Google Colab ini.

In [ ]:
from google.colab import files

print("=== Silakan Upload File 'titanic_clean.csv' ===")
uploaded = files.upload()

## 3. Load & Cek Data
Membaca file CSV yang baru saja diupload ke dalam variabel DataFrame (`df`) dan menampilkan 5 baris pertama untuk memastikan data terbaca dengan benar.

In [ ]:
import io

# Membaca file yang diupload (mengambil nama file secara otomatis)
# Jika dijalankan di lokal/VS Code, cukup gunakan: df = pd.read_csv('titanic_clean.csv')
try:
    filename = next(iter(uploaded))
    df = pd.read_csv(io.BytesIO(uploaded[filename]))
    print(f"\nBerhasil membaca file: {filename}")
except NameError:
    # Fallback jika tidak menggunakan fitur upload colab (file sudah ada di folder)
    df = pd.read_csv('titanic_clean.csv')
    print("\nMembaca file lokal: titanic_clean.csv")

print("=== 5 Baris Pertama Data Titanic ===")
print(df.head())

## 4. Visualisasi Data
Kita akan menggunakan Scatter Plot untuk melihat pola persebaran data. 
Grafik ini menghubungkan **Umur (Age)** dan **Harga Tiket (Fare)**, dengan warna titik yang membedakan antara penumpang yang **Selamat (1)** dan **Tidak Selamat (0)**.

In [ ]:
plt.figure(figsize=(10, 6))
sns.scatterplot(x='Age', y='Fare', hue='Survived', data=df, palette='seismic')
plt.title('Sebaran Penumpang Titanic (Umur vs Harga Tiket)')
plt.xlabel('Umur')
plt.ylabel('Harga Tiket')
plt.legend(title='Status (1=Selamat)')
plt.show()

## 5. Splitting Data (Membagi Data)
Tahap ini memisahkan data menjadi dua bagian:
1.  **Fitur (X)**: Atribut data (Kelas, Gender, Umur, Keluarga, Harga).
2.  **Target (y)**: Label yang ingin diprediksi (Survived).

Selain itu, data juga dibagi menjadi **Data Latih (80%)** untuk melatih model, dan **Data Uji (20%)** untuk mengukur performa model nanti.

In [ ]:
# X = Fitur (Hapus kolom target 'Survived')
X = df.drop('Survived', axis=1)
# y = Target (Hanya kolom 'Survived')
y = df['Survived']

# Bagi data: 80% Training, 20% Testing
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

print(f"Jumlah Data Latih: {len(X_train)}")
print(f"Jumlah Data Uji  : {len(X_test)}")

## 6. Training Model
Kita melatih algoritma **K-Nearest Neighbors (KNN)**. Di sini kita menggunakan parameter `n_neighbors=5`, yang berarti model akan melihat 5 tetangga terdekat untuk menentukan klasifikasi.

In [ ]:
# Inisialisasi model KNN dengan k=5
model = KNeighborsClassifier(n_neighbors=5)

# Latih model menggunakan data training
model.fit(X_train, y_train)
print("\nModel Titanic berhasil dilatih!")

## 7. Evaluasi Model
Setelah dilatih, kita minta model untuk memprediksi data uji (`X_test`). Hasil prediksinya dibandingkan dengan kunci jawaban asli (`y_test`) untuk menghitung akurasi.

In [ ]:
# Prediksi data testing
y_pred = model.predict(X_test)

# Hitung akurasi
akurasi = accuracy_score(y_test, y_pred)
print(f"\n=== AKURASI MODEL: {akurasi * 100:.2f}% ===")

# Tampilkan laporan klasifikasi detail
print("\nLaporan Klasifikasi:")
print(classification_report(y_test, y_pred))

## 8. Simulasi Prediksi Manual
Di sini kita mencoba menguji model dengan data buatan sendiri (data Jack dan Rose) untuk melihat bagaimana model memprediksi kasus spesifik berdasarkan pola yang sudah dipelajari.

In [ ]:
print("\n=== Simulasi Prediksi: Jack vs Rose ===")

# Format Input Sesuai Kolom: [Pclass, Sex, Age, SibSp, Parch, Fare]
# Keterangan: Sex 0 = Pria, 1 = Wanita

# Data Jack: Kelas 3 (Ekonomi), Pria, Umur 20, Sendiri (0 sib, 0 par), Tiket Murah
data_jack = [3, 0, 20, 0, 0, 7.5]

# Data Rose: Kelas 1 (Eksekutif), Wanita, Umur 17, Bawa Ortu (1 par), Tiket Mahal
data_rose = [1, 1, 17, 0, 1, 100.0]

# Buat DataFrame baru untuk inputan ini
input_baru = pd.DataFrame([data_jack, data_rose], columns=X.columns)

# Lakukan prediksi
hasil_prediksi = model.predict(input_baru)

# Tampilkan hasil
for i, nama in enumerate(['Jack', 'Rose']):
    status = "SELAMAT" if hasil_prediksi[i] == 1 else "TIDAK SELAMAT"
    print(f"Prediksi untuk {nama}: {status}")